# 📓 Parameter Search + Fibonacci×Divergence Confluence (TOP/DOWN)
이 노트북은 다음을 한 번에 수행합니다.

1) **볼츠만(softmax/annealing)** 로 RSI / StochRSI / ATR 파라미터 자동 탐색
2) 최적 파라미터를 **적용**해 인디케이터 재계산
3) 최신 스윙의 **피보나치 레벨(센터)** + **ATR 반폭 바운더리** + **RSI 매핑**
4) **일봉(TOP)** + **4시간(DOWN)** 다이버전스 가능 가격/겹침 리포트 출력

> 실행 전 준비물:
> - `df`: **일봉** DataFrame (필수 컬럼: `close, high, low`)
> - `df_`: **4시간** DataFrame (필수 컬럼: `close, high, low`)
> - 인덱스 또는 `datetime` 컬럼이 시간축을 표현해야 합니다.
> - TA-Lib 설치가 필요합니다.


In [1]:
# === Imports & Checks ===
import numpy as np, pandas as pd
try:
    import talib
except Exception as e:
    raise RuntimeError("TA-Lib이 필요합니다. 환경에 TA-Lib을 설치한 뒤 다시 실행하세요.\n"
                       "예: pip install TA-Lib (사전 라이브러리 필요)") from e
print("OK: numpy/pandas/talib loaded")


C:\Users\hahonggu\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\hahonggu\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


OK: numpy/pandas/talib loaded


In [1]:
# 기술적 지표 기본 클래스
class Technical_indicators:
    '''
    기술적 지표 클래스 (바이낸스 기준)
    
    '''
    def __init__(self, ticker, interval, cnt):
        self.ticker = ticker # 티커이름
        self.interval = interval # 인터벌
        self.cnt = cnt # 불러올 테이블 개수
        self.df = Technical_indicators.data_select(self)

    # 데이터 조회
    def data_select(self):
        ohlcv = exchange.fetch_ohlcv(symbol=self.ticker, timeframe=self.interval, limit= self.cnt)
        self.df = pd.DataFrame(ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
        pd_ts = pd.to_datetime(self.df['datetime'], utc=True, unit='ms')     # unix timestamp to pandas Timeestamp
        pd_ts = pd_ts.dt.tz_convert("Asia/Seoul")                       # convert timezone
        pd_ts = pd_ts.dt.tz_localize(None)
        self.df.set_index(pd_ts, inplace=True)
        self.df = self.df[['open', 'high', 'low', 'close', 'volume']]
        self.df.reset_index(inplace=True)
        date = self.df['datetime'].apply(lambda x:x.strftime('%Y-%m-%d'))
        hour = self.df['datetime'].apply(lambda x:x.strftime('%H')+':00:00')
        self.df['date'] = date
        self.df['hour'] = hour
        return self.df
    
    # RSI
    def RSI(self, timeperiod):
        self.df['rsi'] = talib.RSI(self.df['close'],timeperiod= timeperiod)
        self.df['rsi'] = self.df['rsi'].apply(lambda x:round(x,2))
#         self.df.dropna(inplace=True)
        # self.df.iloc[-1] = self.df.iloc[-2]
        return self.df

    # 스토캐스틱 RSI
    def STOCHRSI(self, timeperiod, d, m):
        self.df['close_stoch'] = talib.STOCHRSI(self.df["close"],timeperiod,fastk_period=timeperiod,fastd_period=d,fastd_matype=m)[-1]
        self.df['close_stoch'] = self.df['close_stoch'].apply(lambda x:round(x,2))
#         self.df.dropna(inplace=True)
        return self.df
    
    # ATR 
    def ATR(self, timeperiod):
        self.df['atr'] = talib.ATR(self.df['high'], self.df['low'], self.df['close'], timeperiod=timeperiod)
        return self.df
    
    # EMA 지수이동평균
    def EMA(self, timeperiod):
        self.df['ema'] = talib.EMA(self.df['close'], timeperiod=timeperiod)
        return self.df
    
def zigzag(prices, up_thresh=0.05, down_thresh=-0.05):
    """
    ZigZag 알고리즘 (변곡점 감지)
    prices: list 또는 pandas.Series 형태의 가격 데이터
    up_thresh: 상승 전환 임계값 (예: 0.05 = 5%)
    down_thresh: 하락 전환 임계값 (예: -0.05 = -5%)
    """
    pivots = np.zeros(len(prices))
    last_pivot = prices[0]
    last_pivot_idx = 0
    trend = 0  # 0 = 초기, 1 = 상승, -1 = 하락

    for i in range(1, len(prices)):
        change = (prices[i] - last_pivot) / last_pivot

        if trend == 0:
            if change >= up_thresh:
                trend = 1
                last_pivot = prices[i]
                last_pivot_idx = i
                pivots[i] = 1  # peak
            elif change <= down_thresh:
                trend = -1
                last_pivot = prices[i]
                last_pivot_idx = i
                pivots[i] = -1  # valley

        elif trend == 1:
            if prices[i] > last_pivot:
                last_pivot = prices[i]
                last_pivot_idx = i
            elif change <= down_thresh:
                trend = -1
                pivots[last_pivot_idx] = 1
                last_pivot = prices[i]
                last_pivot_idx = i
                pivots[i] = -1

        elif trend == -1:
            if prices[i] < last_pivot:
                last_pivot = prices[i]
                last_pivot_idx = i
            elif change >= up_thresh:
                trend = 1
                pivots[last_pivot_idx] = -1
                last_pivot = prices[i]
                last_pivot_idx = i
                pivots[i] = 1

    return pivots

import time
from datetime import datetime
import pytz

import pandas as pd
import numpy as np
import talib
import requests
import json
from datetime import datetime, timedelta
import ccxt
from pytz import timezone
import sys
import json

api_key = ''
api_secret = ''


exchange = ccxt.binance(config={
    'apiKey': api_key,
    'secret': api_secret,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future'
    }
})

ticker = 'BTC/USDT'

# interval
top_time = '1w'
down_time = '1d'

cnt=1000

df_top = Technical_indicators(ticker=ticker, interval=top_time, cnt=cnt)
df_down = Technical_indicators(ticker=ticker, interval=down_time, cnt=cnt)

# 초기 세팅 (동적으로 최적을 찾기 떄문에 필요없음)
df_top.STOCHRSI(26,3,0)
df_top.RSI(14)
df_top.EMA(100)
df = df_top.ATR(21)

df_down.STOCHRSI(14,3,0)
df_down.RSI(14)
df_down.EMA(100)
df_ = df_down.ATR(21)

C:\Users\hahonggu\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\hahonggu\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
df

,datetime,open,high,low,close,volume,date,hour,close_stoch,rsi,ema,atr
0,2019-09-02 09:00:00,10000.00,10412.65,10000.00,10391.63,3096.291,2019-09-02,09:00:00,NaN,NaN,NaN,NaN
1,2019-09-09 09:00:00,10316.62,10475.54,9884.31,10302.22,110846.484,2019-09-09,09:00:00,NaN,NaN,NaN,NaN
2,2019-09-16 09:00:00,10302.00,10353.81,9530.02,10023.04,160591.544,2019-09-16,09:00:00,NaN,NaN,NaN,NaN
3,2019-09-23 09:00:00,8061.98,10046.91,7700.67,8041.96,279795.272,2019-09-23,09:00:00,NaN,NaN,NaN,NaN
4,2019-09-30 09:00:00,8042.08,8499.00,7709.01,7852.79,257976.889,2019-09-30,09:00:00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
314,2025-09-08 09:00:00,111075.50,116605.70,110566.50,115210.00,727991.907,2025-09-08,09:00:00,50.97,59.20,80392.959965,8111.406742
315,2025-09-15 09:00:00,115210.00,117875.20,114338.50,115183.90,635999.008,2025-09-15,09:00:00,57.46,59.17,81081.889471,7893.563563
316,2025-09-22 09:00:00,115183.90,115344.60,108566.00,112119.60,771095.215,2025-09-22,09:00:00,56.98,55.54,81696.497600,7840.470060
317,2025-09-29 09:00:00,112119.70,125877.30,111501.00,123426.70,964102.563,2025-09-29,09:00:00,61.28,64.26,82522.838242,8151.700057


## 1) Core helpers — ATR, ZigZag, Fibonacci

In [3]:
# 1) 우리가 쓸 타임프레임만 지원 (5m, 15m, 1h, 4h, 1d)
def make_param_space_for_tf(tf: str):
    tf = tf.lower()
    if tf == '5m':
        up_dn = (0.003, 0.012)   # 0.3% ~ 1.2%
        atrp  = (14, 28)
        mult  = (1.2, 2.2)
        mb    = (3, 8)
    elif tf == '15m':
        up_dn = (0.007, 0.03)    # 0.7% ~ 3.0%
        atrp  = (14, 35)
        mult  = (1.4, 2.6)
        mb    = (5, 12)
    elif tf == '1h':
        up_dn = (0.010, 0.040)   # 1.0% ~ 4.0%
        atrp  = (14, 35)
        mult  = (1.4, 2.8)
        mb    = (6, 12)
    elif tf == '4h':
        up_dn = (0.020, 0.060)   # 2.0% ~ 6.0%
        atrp  = (21, 42)
        mult  = (1.6, 3.0)
        mb    = (7, 14)
    else:  # '1d' 등
        up_dn = (0.030, 0.080)   # 3.0% ~ 8.0%
        atrp  = (21, 42)
        mult  = (1.6, 3.0)
        mb    = (7, 14)

    return {
        # 인디케이터(원하면 범위 줄여도 됨)
        'stochrsi_period': {'type':'int','min':7,'max':34},
        'stochrsi_fastd':  {'type':'int','min':2,'max':6},
        'atr_period':      {'type':'int','min': atrp[0],'max': atrp[1]},
        # 지그재그
        'zig_up_pct':        {'type':'float','min': up_dn[0],'max': up_dn[1]},
        'zig_down_pct':      {'type':'float','min': up_dn[0],'max': up_dn[1]},
        'zig_atr_mult':      {'type':'float','min': mult[0],'max': mult[1]},
        'zig_min_bars':      {'type':'int','min': mb[0],'max':  mb[1]},
        'zig_atr_period':    {'type':'int','min': atrp[0],'max': atrp[1]},
        'zig_threshold_mode':{'type':'choice','choices':['or','and','max']},
    }

# 2) TOP/DOWN 순서 보정 (혹시 순서를 거꾸로 넣어도 TOP=큰 타임프레임으로 고정)
def canonicalize_pair(t1: str, t2: str):
    order = {'5m':0, '15m':1, '1h':2, '4h':3, '1d':4}
    a, b = t1.lower(), t2.lower()
    if order[a] >= order[b]:
        return a, b   # 이미 a가 TOP(더 큼)
    else:
        return b, a   # b를 TOP으로 승격

# 3) 변동성 기반으로 TOP 타임프레임의 up/down 허용범위 미세 조정
def vol_scaled_param_space(df_top, base_tf_space, vol_lookback=300):
    close = df_top['close'].astype(float).to_numpy()
    high  = df_top['high'].astype(float).to_numpy()
    low   = df_top['low'].astype(float).to_numpy()
    prev_close = np.r_[close[0], close[:-1]]
    tr = np.maximum(high-low, np.maximum(np.abs(high-prev_close), np.abs(low-prev_close)))
    rel = tr / np.maximum(close, 1e-9)
    v = float(np.nanmedian(rel[-vol_lookback:]))  # 예: 0.004 = 0.4%

    k_lo, k_hi = 1.2, 3.0
    up_lo = max(base_tf_space['zig_up_pct']['min'],  v*k_lo)
    up_hi = min(base_tf_space['zig_up_pct']['max'],  v*k_hi)
    dn_lo = max(base_tf_space['zig_down_pct']['min'],v*k_lo)
    dn_hi = min(base_tf_space['zig_down_pct']['max'],v*k_hi)

    ps = dict(base_tf_space)
    ps['zig_up_pct']   = {'type':'float','min': up_lo,'max': up_hi}
    ps['zig_down_pct'] = {'type':'float','min': dn_lo,'max': dn_hi}
    return ps
def compute_vol_stats(df, atr_period=21, lookback=400):
    """
    df: 반드시 ['high','low','close','open'] 포함
    반환:
      natr_med   : median(ATR/close)   → %단위 (예: 0.004 = 0.4%)
      body_med   : median(|close-open|/close)
      range_med  : median((high-low)/close)
    """
    d = df.tail(lookback).copy()
    h = d['high'].astype(float).to_numpy()
    l = d['low'].astype(float).to_numpy()
    c = d['close'].astype(float).to_numpy()
    o = d['open'].astype(float).to_numpy()

    # ATR (Wilder) – talib 우선, 실패시 수동
    try:
        atr = talib.ATR(h, l, c, timeperiod=int(atr_period))
    except Exception:
        prev_close = np.r_[c[0], c[:-1]]
        tr = np.maximum(h-l, np.maximum(np.abs(h-prev_close), np.abs(l-prev_close)))
        atr = pd.Series(tr).ewm(alpha=1/atr_period, adjust=False).mean().values

    natr = atr / np.maximum(c, 1e-9)
    body = np.abs(c - o) / np.maximum(c, 1e-9)
    rng  = (h - l)      / np.maximum(c, 1e-9)

    # robust central tendency
    natr_med  = float(np.nanmedian(natr))
    body_med  = float(np.nanmedian(body))
    range_med = float(np.nanmedian(rng))
    return natr_med, body_med, range_med

def atr_scaled_param_space(df_top, base_space,
                           atr_period=21,
                           lookback=400,
                           # up/down 퍼센트 범위를 nATR 기반으로 재설정할 때 곱해줄 계수
                           updn_k_low=1.4, updn_k_high=3.2,
                           # atr_mult도 nATR에 따라 미세조정 (기본범위 안에서)
                           mult_bias=1.0, mult_gamma=0.0,
                           # min_bars도 변동성에 따라 살짝 조정
                           min_bars_lo_shift=-2, min_bars_hi_shift=+2):
    """
    - base_space: make_param_space_for_tf(top_tf) 결과(dict)
    - nATR가 높을수록 up/down 임계치 범위를 넓히고(더 큰 변동 허용),
      nATR가 낮을수록 범위를 좁힌다.
    - atr_mult는 보수적으로: mult_bias * (nATR / ref)^gamma (gamma=0이면 고정)
    - min_bars는 변동성 높으면 소폭 증가(+), 낮으면 소폭 감소(-) 정도로만.
    """
    ps = {k: (v.copy() if isinstance(v, dict) else v) for k, v in base_space.items()}

    # 1) 변동성 통계
    natr_med, body_med, range_med = compute_vol_stats(df_top, atr_period=atr_period, lookback=lookback)
    # 하나의 스칼라로 요약 (원하면 가중치 조정 가능)
    vol_score = 0.6 * natr_med + 0.2 * body_med + 0.2 * range_med   # 0.0~?
    # 기준 nATR (타임프레임별 대략값; 필요시 tf별 상수로 관리 가능)
    ref_natr = 0.004  # 0.4% 정도를 기준치로 가정
    scale = np.clip(vol_score / max(ref_natr, 1e-9), 0.5, 2.5)  # 보호용 클램프

    # 2) zig_up_pct / zig_down_pct 범위 재설정
    #    base_min/base_max를 기준으로 nATR*계수 범위를 CLAMP
    def _reshape_updn(key):
        lo0, hi0 = ps[key]['min'], ps[key]['max']
        lo = max(lo0, vol_score * updn_k_low)
        hi = min(hi0, vol_score * updn_k_high)
        if hi < lo:  # 극단적 케이스 보호
            mid = (lo0 + hi0)/2
            lo, hi = mid*0.8, mid*1.2
        ps[key]['min'], ps[key]['max'] = lo, hi

    _reshape_updn('zig_up_pct')
    _reshape_updn('zig_down_pct')

    # 3) zig_atr_mult 범위도 소폭 보정 (선택)
    #    mult ≈ base * (scale^gamma), gamma=0이면 그대로
    if 'zig_atr_mult' in ps and mult_gamma != 0.0:
        lo0, hi0 = ps['zig_atr_mult']['min'], ps['zig_atr_mult']['max']
        base_mid = (lo0 + hi0)/2
        adj_mid  = mult_bias * base_mid * (scale ** mult_gamma)
        # mid 주변으로 ±20% 정도의 창을 주고 base 범위와 교집합
        lo = max(lo0, adj_mid * 0.8)
        hi = min(hi0, adj_mid * 1.2)
        if hi > lo:
            ps['zig_atr_mult']['min'], ps['zig_atr_mult']['max'] = lo, hi

    # 4) min_bars 약간 조정 (변동성 높으면 더 신중하게)
    if 'zig_min_bars' in ps:
        lo0, hi0 = ps['zig_min_bars']['min'], ps['zig_min_bars']['max']
        # scale>1 → 변동성 높음 → hi 쪽으로 살짝 이동
        lo = int(np.clip(lo0 + min_bars_lo_shift, 1, hi0))
        hi = int(np.clip(hi0 + min_bars_hi_shift*min(scale,1.5), lo, hi0+5))
        ps['zig_min_bars']['min'], ps['zig_min_bars']['max'] = lo, hi

    return ps, dict(natr_med=natr_med, body_med=body_med, range_med=range_med, vol_score=vol_score)

def vol_scaled_param_space(df, base_tf_space, vol_lookback=300):
    close = df['close'].astype(float).to_numpy()
    high  = df['high'].astype(float).to_numpy()
    low   = df['low'].astype(float).to_numpy()

    prev_close = np.r_[close[0], close[:-1]]
    tr = np.maximum(high-low, np.maximum(np.abs(high-prev_close), np.abs(low-prev_close)))
    rel = tr / np.maximum(close, 1e-9)
    v = float(np.nanmedian(rel[-vol_lookback:]))  # 예: 0.004 = 0.4%

    k_lo, k_hi = 1.2, 3.0  # 변동성 배수 밴드
    up_lo = max(base_tf_space['zig_up_pct']['min'],  v*k_lo)
    up_hi = min(base_tf_space['zig_up_pct']['max'],  v*k_hi)
    dn_lo = max(base_tf_space['zig_down_pct']['min'],v*k_lo)
    dn_hi = min(base_tf_space['zig_down_pct']['max'],v*k_hi)

    ps = dict(base_tf_space)
    ps['zig_up_pct']   = {'type':'float','min': up_lo,'max': up_hi}
    ps['zig_down_pct'] = {'type':'float','min': dn_lo,'max': dn_hi}
    return ps


In [4]:
def wilder_atr(high, low, close, period=14):
    high = np.asarray(high, dtype=float)
    low = np.asarray(low, dtype=float)
    close = np.asarray(close, dtype=float)
    prev_close = np.r_[close[0], close[:-1]]
    tr = np.maximum(high - low, np.maximum(np.abs(high - prev_close), np.abs(low - prev_close)))
    atr = pd.Series(tr).ewm(alpha=1/period, adjust=False).mean().values
    return atr

def zigzag_meaningful_v2(
    close, high=None, low=None,
    up_pct=0.05, down_pct=0.05,
    atr_period=14, atr_mult=2.0,
    threshold_mode='or',          # 'or' | 'and' | 'max'
    use_hl=True,
    min_bars=5,
    min_swing_atr=1.0,
    finalize_last=False
):
    price = np.asarray(close, dtype=float)
    if use_hl:
        if high is None or low is None:
            raise ValueError("use_hl=True면 high/low가 필요합니다.")
        high = np.asarray(high, dtype=float)
        low  = np.asarray(low, dtype=float)
    else:
        high = price; low = price

    # ATR
    try:
        atr = talib.ATR(high.astype(float), low.astype(float), price.astype(float), timeperiod=int(atr_period))
    except Exception:
        atr = wilder_atr(high, low, price, period=int(atr_period))

    def moved_up(ext_val, i):
        pct_ok = (high[i] - ext_val) / max(ext_val, 1e-12) >= up_pct
        atr_ok = (high[i] - ext_val) >= atr_mult * atr[i]
        if threshold_mode == 'and': return pct_ok and atr_ok
        if threshold_mode == 'max': return (high[i] - ext_val) >= max(up_pct*ext_val, atr_mult*atr[i])
        return pct_ok or atr_ok

    def moved_down(ext_val, i):
        pct_ok = (ext_val - low[i]) / max(ext_val, 1e-12) >= down_pct
        atr_ok = (ext_val - low[i]) >= atr_mult * atr[i]
        if threshold_mode == 'and': return pct_ok and atr_ok
        if threshold_mode == 'max': return (ext_val - low[i]) >= max(down_pct*ext_val, atr_mult*atr[i])
        return pct_ok or atr_ok

    n = len(price)
    pivots = np.zeros(n, dtype=int)
    trend = 0
    ext_idx = 0
    ext_val = price[0]

    for i in range(1, n):
        if trend == 0:
            if moved_up(ext_val, i):
                trend = 1; ext_idx = i; ext_val = high[i]; continue
            if moved_down(ext_val, i):
                trend = -1; ext_idx = i; ext_val = low[i];  continue
        elif trend == 1:
            if high[i] > ext_val:
                ext_idx, ext_val = i, high[i]
            if moved_down(ext_val, i):
                pivots[ext_idx] = 1; trend = -1; ext_idx, ext_val = i, low[i]
        else:
            if low[i] < ext_val:
                ext_idx, ext_val = i, low[i]
            if moved_up(ext_val, i):
                pivots[ext_idx] = -1; trend = 1; ext_idx, ext_val = i, high[i]

    if finalize_last and trend != 0:
        pivots[ext_idx] = 1 if trend == 1 else -1

    # Pruning: alternate & min swing
    def extreme_at(idx, sign): return high[idx] if sign == 1 else low[idx]
    idxs = np.where(pivots != 0)[0].tolist()
    j = 1
    while j < len(idxs):
        a, b = idxs[j-1], idxs[j]
        if pivots[a] == pivots[b]:
            if pivots[a] == 1:
                keep = a if high[a] >= high[b] else b
            else:
                keep = a if low[a]  <= low[b]  else b
            drop = b if keep == a else a
            pivots[drop] = 0
            idxs.pop(j if drop == b else j-1)
            j = max(1, j-1)
        else:
            j += 1

    changed = True
    while changed and len(idxs) >= 3:
        changed = False
        k = 1
        while k < len(idxs)-1:
            a, b, c = idxs[k-1], idxs[k], idxs[k+1]
            sign_b = pivots[b]
            if (b - a) < min_bars or (c - b) < min_bars:
                pivots[b] = 0; idxs.pop(k); changed = True; continue
            amp1 = abs(extreme_at(b, sign_b) - extreme_at(a, pivots[a]))
            amp2 = abs(extreme_at(c, pivots[c]) - extreme_at(b, sign_b))
            thr  = min_swing_atr * atr[b]
            if min(amp1, amp2) < thr:
                pivots[b] = 0; idxs.pop(k); changed = True; continue
            k += 1
    return pivots, atr

def latest_alternating_pivots(df, pivot_col='pivot', pivot_price_col='pivot_price'):
    pvals = df[pivot_col].to_numpy()
    idxs = np.flatnonzero(pvals != 0)
    if len(idxs) < 2:
        raise ValueError("확정 피벗이 2개 이상 필요합니다.")
    for k in range(len(idxs)-1, 0, -1):
        i2 = idxs[k]; i1 = idxs[k-1]
        s1, s2 = int(pvals[i1]), int(pvals[i2])
        if s1 == s2: 
            continue
        p1 = float(df[pivot_price_col].iat[i1])
        p2 = float(df[pivot_price_col].iat[i2])
        if s1 == -1 and s2 == 1: direction = 'up'
        elif s1 == 1 and s2 == -1: direction = 'down'
        else: continue
        return {'start_pos': i1, 'end_pos': i2, 'start_ts': df.index[i1], 'end_ts': df.index[i2],
                'start_sign': s1, 'end_sign': s2, 'start_price': p1, 'end_price': p2, 'direction': direction}
    raise ValueError("교대(고↔저) 피벗 쌍을 찾을 수 없습니다.")

def fib_from_latest(df, pivot_col='pivot', pivot_price_col='pivot_price',
                    retracements=(0.236, 0.382, 0.5, 0.618, 0.786),
                    extensions=(1.272, 1.414, 1.618),
                   include_extremes=True):
    swing = latest_alternating_pivots(df, pivot_col, pivot_price_col)
    if swing['direction'] == 'up':
        lo, hi = swing['start_price'], swing['end_price']
        anchors = {'0': hi, '1': lo}
        rets = {r: hi - (hi - lo) * r for r in retracements}
        exts = {e: hi + (hi - lo) * e for e in extensions}
        gp   = (hi - (hi - lo)*0.65, hi - (hi - lo)*0.618)
    else:
        hi, lo = swing['start_price'], swing['end_price']
        anchors = {'0': lo, '1': hi}
        rets = {r: lo + (hi - lo) * r for r in retracements}
        exts = {e: lo - (hi - lo) * e for e in extensions}
        gp   = (lo + (hi - lo)*0.618, lo + (hi - lo)*0.65)
    # R0/R100 자동 주입 (원하면 끌 수 있음)
    if include_extremes:
        rets = {**rets, 0.0: float(anchors['0']), 1.0: float(anchors['1'])}
    gp = (min(gp), max(gp))
    return {'swing': swing, 'anchors': anchors, 'retracements': rets, 'extensions': exts, 'golden_pocket': gp}


## 2) Divergence & Boundary tools

In [5]:
def _ts_series(df):
    return df['datetime'] if 'datetime' in df.columns else df.index

def pick_oversold_segment_D_with_current_rule(df, d_col='close_stoch', oversold=20.0, auto_scale=True):
    d = df[d_col].astype(float).to_numpy()
    if auto_scale:
        maxv = np.nanmax(d)
        thr = oversold/100.0 if maxv <= 1.0 else oversold
    else:
        thr = oversold
    n = len(d); segs = []; i = n-1
    while i >= 0:
        if np.isfinite(d[i]) and d[i] <= thr:
            b = i; a = i
            while a-1 >= 0 and np.isfinite(d[a-1]) and d[a-1] <= thr:
                a -= 1
            segs.append((a,b)); i = a - 1
        else:
            i -= 1
    segs = segs[::-1]
    if not segs: return (None, thr, 'no_segment')
    cur = d[-1]
    if np.isfinite(cur) and cur <= thr:
        return (segs[-2], thr, 'current_oversold_use_previous') if len(segs)>=2 else (None, thr, 'current_oversold_but_no_previous')
    else:
        return (segs[-1], thr, 'current_not_oversold_use_latest')

def ref_from_segment_min_close(df, seg, *, rsi_col='rsi'):
    a, b = seg; sub = df.iloc[a:b+1]
    idx_min = sub['close'].idxmin()
    iloc_min = df.index.get_loc(idx_min)
    return {'ref_idx': iloc_min, 'ref_ts': idx_min,
            'ref_price': float(df.at[idx_min, 'close']),
            'ref_rsi': float(df.at[idx_min, rsi_col])}

def needed_close_regular_now(df, *, ref_price, ref_rsi, rsi_period=14,
                             lower_bound=None, eps=1e-8, tol=1e-6, max_iter=60):
    close = df['close'].to_numpy(float).copy()
    def rsi_last(x):
        close[-1] = x
        return talib.RSI(close, timeperiod=int(rsi_period))[-1]
    U = ref_price - max(eps, abs(ref_price)*1e-6)
    if lower_bound is None:
        L = U - max(1e-6, abs(U)*0.10)
    else:
        L = min(lower_bound, U - max(1e-6, abs(U)*0.001))
    if not np.isfinite(U) or L >= U:
        return None
    rU = rsi_last(U)
    if (not np.isfinite(rU)) or rU <= ref_rsi:
        return None
    lo, hi = L, U
    for _ in range(max_iter):
        mid = (lo + hi)/2.0
        rmid = rsi_last(mid)
        if not np.isfinite(rmid): lo = mid; continue
        if rmid > ref_rsi: hi = mid
        else: lo = mid
        if abs(hi - lo) <= tol: break
    return float(min(hi, U))

def feasible_close_range_hidden_now(df, *, ref_price, ref_rsi, rsi_period=14,
                                    upper_bound=None, eps=1e-8, tol=1e-6, max_iter=60):
    close = df['close'].to_numpy(float).copy()
    def rsi_last(x):
        close[-1] = x
        return talib.RSI(close, timeperiod=int(rsi_period))[-1]
    L = ref_price + max(eps, abs(ref_price)*1e-6)
    if upper_bound is None:
        U = L + max(1e-6, abs(L)*0.10)
    else:
        U = max(upper_bound, L + max(1e-6, abs(L)*0.001))
    if not np.isfinite(L) or L >= U:
        return None
    rL = rsi_last(L)
    if (not np.isfinite(rL)) or rL >= ref_rsi:
        return None
    lo, hi = L, U
    for _ in range(max_iter):
        mid = (lo + hi)/2.0
        rmid = rsi_last(mid)
        if not np.isfinite(rmid): hi = mid; continue
        if rmid < ref_rsi: lo = mid
        else: hi = mid
        if abs(hi - lo) <= tol: break
    xmax = float(lo)
    if xmax <= L: return None
    return (float(L), xmax)

def _rsi_at_price(df, price, rsi_period=14):
    close = df['close'].astype(float).to_numpy().copy()
    close[-1] = float(price)
    r = talib.RSI(close, timeperiod=int(rsi_period))
    return float(r[-1])

def _atr_at_end(df, res, atr_col='atr'):
    end_pos = res['swing']['end_pos']
    return float(pd.Series(df[atr_col]).iloc[:end_pos+1].dropna().iloc[-1])

def build_fib_ratio_boundaries(df, res, *,
                               atr_col='atr',
                               ratios=(0.236, 0.382, 0.5, 0.618, 0.786),
                               k=1.0, mode='linear', gamma=1.0,
                               max_half_mult=None):
    atr_end = _atr_at_end(df, res, atr_col=atr_col)
    rets = res['retracements']
    def _nearest_key(x): return min(rets.keys(), key=lambda kk: abs(float(kk)-float(x)))
    def g(r):
        if mode == 'sqrt': return np.sqrt(r)
        if mode == 'pow' : return r**gamma
        return r
    bounds = []
    for r in ratios:
        rk = r if r in rets else _nearest_key(r)
        lvl  = float(rets[rk])
        half = float(k * g(float(r)) * atr_end)
        if max_half_mult is not None:
            half = min(half, float(max_half_mult * atr_end))
        bounds.append({'ratio': float(r), 'label': f'R{int(round(float(r)*100))}',
                       'level': lvl, 'low': lvl - half, 'high': lvl + half, 'half': half})
    bounds.sort(key=lambda x: x['level'])
    return bounds, atr_end

def make_level_map_from_res(res, ratios=(0.786,0.618,0.5,0.382,0.236)):
    rets = res['retracements']
    def _nearest_key(x): return min(rets.keys(), key=lambda k: abs(float(k)-float(x)))
    m = {}
    for r in ratios:
        rk = r if r in rets else _nearest_key(r)
        m[f"R{int(round(r*100))}"] = float(rets[rk])
    return m

def print_level_divergence_hits(df, bounds, *, need_reg=None, hid_range=None,
                                rsi_period=14, level_map=None, show_center=True):
    import numpy as np
    if need_reg is not None:
        try: need_reg = float(need_reg)
        except: need_reg = None
    if hid_range is not None:
        try:
            Lh, Xh = float(hid_range[0]), float(hid_range[1])
            if not (np.isfinite(Lh) and np.isfinite(Xh)): hid_range = None
            else: hid_range = (Lh, Xh)
        except: hid_range = None

    def _rsi_range_on_interval(df, L, H, rsi_period=14, samples=5):
        L = float(L); H = float(H)
        if not (np.isfinite(L) and np.isfinite(H)) or H <= L:
            return (None, None)
        xs = np.linspace(L, H, max(2, int(samples)))
        vals = [ _rsi_at_price(df, x, rsi_period) for x in xs ]
        vals = [ v for v in vals if np.isfinite(v) ]
        if not vals: return (None, None)
        return (float(np.min(vals)), float(np.max(vals)))

    print("\n=== LEVEL × DIVERGENCE CHECK ===")
    for b in bounds:
        lbl, low, high = b['label'], float(b['low']), float(b['high'])
        ctr = float(level_map[lbl]) if (level_map and lbl in level_map) else float(b.get('level', (low+high)/2))
        rsi_lo = _rsi_at_price(df, low,  rsi_period)
        rsi_hi = _rsi_at_price(df, high, rsi_period)
        print(f"{lbl}: Price {ctr:,.2f} -> [{low:,.2f} ~ {high:,.2f}] | RSI[{rsi_lo:.2f} ~ {rsi_hi:.2f}]")
        print(f"   · RSI→Price (boundary): {rsi_lo:.2f}→{low:,.2f} , {rsi_hi:.2f}→{high:,.2f}")
        # Regular
        reg_hit, reg_rng = False, None
        if need_reg is not None and np.isfinite(need_reg):
            hi = min(high, need_reg)
            if hi > low: reg_hit, reg_rng = True, (low, hi)
        # Hidden
        hid_hit, hid_rng = False, None
        if hid_range is not None:
            Lh, Xh = hid_range
            lo = max(low, Lh); hi = min(high, Xh)
            if hi > lo: hid_hit, hid_rng = True, (lo, hi)
        # BOTH
        both_rng = None
        if reg_rng and hid_rng:
            lo = max(reg_rng[0], hid_rng[0]); hi = min(reg_rng[1], hid_rng[1])
            if hi > lo: both_rng = (lo, hi)
        # Print
        if reg_rng:
            Rmin, Rmax = _rsi_range_on_interval(df, reg_rng[0], reg_rng[1], rsi_period)
            print(f"   ★ Regular : YES  overlap={reg_rng[0]:,.2f}~{reg_rng[1]:,.2f}  |  RSI@Regular[{Rmin:.2f} ~ {Rmax:.2f}]")
        else:
            why = ("need_reg=None" if need_reg is None else (f"need_reg≤low ({need_reg:,.2f}≤{low:,.2f})" if need_reg <= low else "no-overlap"))
            print(f"   - Regular : no  ({why})")
        if hid_rng:
            Rmin, Rmax = _rsi_range_on_interval(df, hid_rng[0], hid_rng[1], rsi_period)
            print(f"   ★ Hidden  : YES  overlap={hid_rng[0]:,.2f}~{hid_rng[1]:,.2f}  |  RSI@Hidden [{Rmin:.2f} ~ {Rmax:.2f}]")
        else:
            print("   - Hidden  : no  (no-overlap)")
        if both_rng:
            Rmin, Rmax = _rsi_range_on_interval(df, both_rng[0], both_rng[1], rsi_period)
            print(f"   - BOTH    : {both_rng[0]:,.2f} ~ {both_rng[1]:,.2f}  |  RSI@BOTH   [{Rmin:.2f} ~ {Rmax:.2f}]")


## 3) Parameterization — Indicators & Boltzmann search

In [6]:
def ensure_stochD_inplace(df, col='close_stoch',
                          rsi_period=14, stochrsi_period=14, fastd=3, fastk=3,
                          force=False):
    # force=True 이면 무조건 재계산
    need = force or (col not in df.columns) or df[col].isna().all()

    # 파라미터가 바뀌면 재계산하도록 안전장치(옵션)
    prev = df.attrs.get('_stoch_params', None)
    curp = (int(stochrsi_period), int(fastd), int(fastk))
    if (not need) and prev is not None and prev != curp:
        need = True

    if not need:
        return

    close = df['close'].astype(float).values
    _, d = talib.STOCHRSI(close,
                          timeperiod=int(stochrsi_period),
                          fastk_period=int(fastk),
                          fastd_period=int(fastd),
                          fastd_matype=0)
    if np.nanmax(d) <= 1.0:
        d = d * 100.0
    df[col] = d
    df.attrs['_stoch_params'] = curp

In [7]:


# =========================
# 1) 샘플러(choice/int/float 지원)
# =========================
def _sample_params(param_space, around=None, scale=0.35, rng=None):
    rng = np.random.default_rng() if rng is None else rng
    out = {}
    for k, spec in param_space.items():
        t = spec.get('type', 'float')
        if t == 'choice':
            choices = spec['choices']
            if (around is None) or (k not in around) or (rng.random() < 0.25):
                v = rng.choice(choices)
            else:
                v = around[k]
        elif t == 'int':
            lo, hi = int(spec['min']), int(spec['max'])
            if (around is None) or (k not in around):
                v = int(rng.integers(lo, hi+1))
            else:
                mu = int(around[k]); sigma = max(1, int((hi-lo)*scale))
                v = int(np.clip(int(rng.normal(mu, sigma)), lo, hi))
        else:  # float
            lo, hi = float(spec['min']), float(spec['max'])
            if (around is None) or (k not in around):
                v = float(rng.uniform(lo, hi))
            else:
                mu = float(around[k]); sigma = max(1e-4, (hi-lo)*scale)
                v = float(np.clip(rng.normal(mu, sigma), lo, hi))
        out[k] = v
    return out


# =========================
# 2) 후보 적용 & 점수 계산
#    (네가 이미 정의해둔 함수들 사용)
#    - zigzag_meaningful_v2
#    - fib_from_latest
#    - build_fib_ratio_boundaries
#    - pick_oversold_segment_D_with_current_rule
#    - ref_from_segment_min_close
#    - needed_close_regular_now
#    - feasible_close_range_hidden_now
#    - fib_divergence_confluence
# =========================
def _apply_candidate_and_score(df, cand, *, zig_params, sample_last=300, step=10,
                               fib_ratios=(0.786,0.618,0.5,0.382,0.236),
                               fib_k=1.0, fib_mode='linear', fib_max_half_mult=1.2):
    # 0) 샘플링 안전장치
    N = min(len(df), int(sample_last))
    if N < 120:
        # 데이터가 너무 짧으면 스킵
        return -1e9
    slc = df.iloc[-N::int(step)].copy()

    # 1) RSI(14) 고정 보장
    if 'rsi' not in slc.columns or slc['rsi'].isna().all():
        slc['rsi'] = talib.RSI(slc['close'].to_numpy(float), timeperiod=14)

    # 2) ATR: 후보 반영
    atr_period = int(cand.get('atr_period', 21))
    slc['atr'] = talib.ATR(
        slc['high'].to_numpy(float),
        slc['low'].to_numpy(float),
        slc['close'].to_numpy(float),
        timeperiod=atr_period
    )

    # 3) StochRSI D: 네가 df에 이미 넣어뒀어도 NaN이면 자동 보정
    #    (후보가 stochrsi_period/fastd를 제시하면 그 값으로 재계산)
    sp = int(cand.get('stochrsi_period', 14))
    fd = int(cand.get('stochrsi_fastd', 3))
    ensure_stochD_inplace(
        slc, col='close_stoch',
        rsi_period=14,
        stochrsi_period=sp, fastd=fd,
        force=True          # ★ 여기 추가
    )

    # 4) ZigZag: 후보로 override
    zp = dict(zig_params or {})
    if 'zig_up_pct'         in cand: zp['up_pct']        = float(cand['zig_up_pct'])
    if 'zig_down_pct'       in cand: zp['down_pct']      = float(cand['zig_down_pct'])
    if 'zig_atr_mult'       in cand: zp['atr_mult']      = float(cand['zig_atr_mult'])
    if 'zig_min_bars'       in cand: zp['min_bars']      = int(cand['zig_min_bars'])
    if 'zig_atr_period'     in cand: zp['atr_period']    = int(cand['zig_atr_period'])
    if 'zig_threshold_mode' in cand: zp['threshold_mode']= str(cand['zig_threshold_mode'])

    try:
        pivots, _ = zigzag_meaningful_v2(
            close=slc['close'], high=slc['high'], low=slc['low'],
            up_pct=zp.get('up_pct',0.04), down_pct=zp.get('down_pct',0.04),
            atr_period=zp.get('atr_period',21), atr_mult=zp.get('atr_mult',1.8),
            threshold_mode=zp.get('threshold_mode','max'),
            use_hl=True, min_bars=zp.get('min_bars',7),
            min_swing_atr=1.0, finalize_last=False
        )
        slc['pivot'] = pivots
        slc['pivot_price'] = np.where(slc['pivot']==1, slc['high'],
                               np.where(slc['pivot']==-1, slc['low'], np.nan))

        # 5) 피보나치/존
        res = fib_from_latest(slc, pivot_col='pivot', pivot_price_col='pivot_price')
        bounds, _ = build_fib_ratio_boundaries(
            slc, res, atr_col='atr',
            ratios=fib_ratios, k=fib_k, mode=fib_mode, max_half_mult=fib_max_half_mult
        )
        zones = [{'low':b['low'], 'high':b['high'], 'strength':1} for b in bounds]

        # 6) 직전 규칙 기반 오버솔드 세그먼트 → ref → 필요한 가격
        seg, _, _ = pick_oversold_segment_D_with_current_rule(
            slc, d_col='close_stoch', oversold=20, auto_scale=True
        )
        if seg is None:
            return -1e9
        ref = ref_from_segment_min_close(slc, seg, rsi_col='rsi')

        need_reg  = needed_close_regular_now(
            slc, ref_price=ref['ref_price'], ref_rsi=ref['ref_rsi'], rsi_period=14
        )
        hid_range = feasible_close_range_hidden_now(
            slc, ref_price=ref['ref_price'], ref_rsi=ref['ref_rsi'], rsi_period=14
        )

        # 7) 컨플루언스 점수
        rows = fib_divergence_confluence(
            slc, res, zones, need_reg=need_reg, hid_range=hid_range, atr_col='atr', sort_by='score'
        )
        if not rows:
            return -1e9
        score = float(np.mean([r['score'] for r in rows]))
        
        # === 추가: 과대/과소 스윙 패널티 ===
        try:
            swing = res['swing']
            swing_amp = abs(swing['end_price'] - swing['start_price'])
            # ATR_end 가져오는 너의 헬퍼가 있으면 사용, 없으면 아래처럼 추출
            end_pos = swing['end_pos']
            atr_end = float(pd.Series(slc['atr']).iloc[:end_pos+1].dropna().iloc[-1])
            ratio = swing_amp / max(atr_end, 1e-9)

            # 타임프레임에 맞는 권장 밴드(예시). 외부에서 인자로 받아도 됨.
            # 5m/15m: 8~25, 1h: 6~18, 4h/1d: 5~15
            tf = str(slc.index.freq) if hasattr(slc.index, 'freq') else '15m'  # 없으면 호출부에서 넘겨도 됨
            lo, hi = 8.0, 25.0  # 기본(15m 예시)
            # 필요시 tf에 따라 분기
            # if top_time in ('5m','15m'): lo, hi = 8.0, 25.0
            # elif top_time in ('1h',):    lo, hi = 6.0, 18.0
            # else:                         lo, hi = 5.0, 15.0

            if ratio < lo:
                penalty = (lo - ratio) / lo
            elif ratio > hi:
                penalty = (ratio - hi) / hi
            else:
                penalty = 0.0

            score -= 0.5 * penalty  # 가중치(0.3~1.0 사이에서 조정)
        except Exception:
            pass
        if not np.isfinite(score):
            return -1e9
        return score

    except Exception:
        # 실패 시 페널티
        return -1e9

def _score_candidate(df, cand, *, windows, zig_params, fib_cfg, param_space, prior=None, stick_lambda=0.0):
    """
    여러 윈도우에서 평균 점수 + 이전 best와의 거리(스틱니스) 패널티를 적용한 최종 점수.
    fib_cfg: {'fib_ratios':..., 'fib_k':..., 'fib_mode':..., 'fib_max_half_mult':...}
    """
    scores = []
    for w in windows:
        s = _score_candidate(
            df, cand,
            windows=windows,
            zig_params=zig_params,
            fib_cfg=fib_cfg,
            param_space=param_space,
            prior=prior,
            stick_lambda=stick_lambda
        )
        scores.append(s)

    base = float(np.mean(scores))

    # --- 스틱니스(이전 best와 너무 멀면 패널티) ---
    if prior:
        dist = 0.0
        for k, spec in param_space.items():
            if (k in cand) and (k in prior):
                # choice 타입은 거리계산 패스
                if spec.get('type') == 'choice':
                    continue
                if ('min' in spec) and ('max' in spec):
                    rng = (spec['max'] - spec['min']) or 1.0
                    dist += ((float(cand[k]) - float(prior[k])) / rng) ** 2
        base -= stick_lambda * dist

    return base

# =========================
# 3) 볼츠만 탐색 본체
# =========================
def boltzmann_search_confluence(df, param_space,
                                iters=15, batch=10, T0=0.6, T_min=0.08,
                                sample_last=300, step=10,
                                fib_ratios=(0.786,0.618,0.5,0.382,0.236),
                                fib_k=1.0, fib_mode='linear', fib_max_half_mult=1.2,
                                zig_params=None,
                                windows=None, seed=None, prior=None, stick_lambda=0.0):
    rng = np.random.default_rng(seed)
    # >>> 추가: 지역 변수 초기화 <<<
    best = None              # (params_dict, meta_dict) 형태로 저장할 예정
    hist = []                # [(params_dict, {'score': ...}), ...]
    best_score = -1e9        # 현재까지 최고 점수
    if not windows:
        windows = [{'sample_last': sample_last, 'step': step}]
    fib_cfg = dict(fib_ratios=fib_ratios, fib_k=fib_k, fib_mode=fib_mode, fib_max_half_mult=fib_max_half_mult)

    for it in range(1, iters+1):
        # 후보군
        if best is None:
            cands = [_sample_params(param_space, around=None, scale=0.35, rng=rng) for _ in range(batch)]
        else:
            around = best[0]  # best = (params, meta)
            cands  = [_sample_params(param_space, around=around, scale=0.25, rng=rng) for _ in range(batch)]

        batch_scores = []
        for cand in cands:
            s = _apply_candidate_and_score(
                df, cand, zig_params=zig_params, sample_last=sample_last, step=step,
                fib_ratios=fib_ratios, fib_k=fib_k, fib_mode=fib_mode, fib_max_half_mult=fib_max_half_mult
            )
            batch_scores.append((cand, {'score': s}))

        # 정렬 및 베스트 갱신
        batch_scores.sort(key=lambda x: x[1]['score'], reverse=True)
        if (best is None) or (batch_scores[0][1]['score'] > best[1]['score']):
            best = batch_scores[0]
        hist.extend(batch_scores)

        # 로그
        T = max(T_min, T0 * (0.85 ** (it-1)))
        batch_avg = float(np.mean([m['score'] for _, m in batch_scores]))
        top_prob  = min(1.0, np.exp((batch_scores[0][1]['score'] - batch_avg) / max(1e-6, T)))
        print(f"[Iter {it}/{iters}] T={T:.3f}  best_total={best[1]['score']:.3f}  batch_avg={batch_avg:.3f}  top_prob={top_prob:.2f}")

    # >>> 추가: 안전 정렬/리턴 <<<
    hist_sorted = sorted(hist, key=lambda x: x[1]['score'], reverse=True) if hist else []
    if best is None:
        best = ({}, {'score': -1e9})  # 아무 후보도 채택 못했을 때의 안전값
    return best, hist_sorted


# =========================
# 4) 최종 적용용: 인디케이터/지그재그 생성기
# =========================
def _apply_params_inplace(df, *, rsi_period=14, atr_period=21,
                          maybe_recompute_stoch=False, stochrsi_period=14, stochrsi_fastd=3):
    df['rsi'] = talib.RSI(df['close'].astype(float).values, timeperiod=int(rsi_period))
    df['atr'] = talib.ATR(df['high'].values, df['low'].values, df['close'].values,
                          timeperiod=int(atr_period))
    if maybe_recompute_stoch:
        fk = 3
        _, d = talib.STOCHRSI(
            df['close'].astype(float).values,
            timeperiod=int(stochrsi_period), fastk_period=fk, fastd_period=int(stochrsi_fastd), fastd_matype=0
        )
        if np.nanmax(d) <= 1.0:
            d = d * 100.0
        df['close_stoch'] = d

def _apply_zigzag_inplace(df,
                          up_pct, down_pct,
                          atr_period, atr_mult,
                          threshold_mode,
                          min_bars,
                          min_swing_atr=1.0,
                          finalize_last=False):
    """
    df에 pivot / pivot_price를 생성.
    """
    pivots, _ = zigzag_meaningful_v2(
        close=df['close'], high=df['high'], low=df['low'],
        up_pct=float(up_pct), down_pct=float(down_pct),
        atr_period=int(atr_period), atr_mult=float(atr_mult),
        threshold_mode=str(threshold_mode),
        use_hl=True,
        min_bars=int(min_bars),
        min_swing_atr=float(min_swing_atr),
        finalize_last=bool(finalize_last)
    )
    df['pivot'] = pivots
    df['pivot_price'] = np.where(df['pivot']==1, df['high'],
                         np.where(df['pivot']==-1, df['low'], np.nan))


# =========================
# 5) SelfTuner
# =========================
class SelfTuner:
    def __init__(self,
                 param_space,
                 zig_params=None,
                 FIB_CFG=None,
                 search_cfg=None,
                 iters=15, batch=10, T0=0.6, T_min=0.08,
                 retune_every=50, vol_window=50, vol_jump=0.35,
                 stoch_col='close_stoch', oversold_thr=20, auto_scale=True,
                 seed=42,
                 stability=None):

        self.param_space   = param_space
        # ↓ 두 줄 추가/수정 (둘 중 하나만 써도 되지만 통일을 위해 둘 다 세팅)
        self.zig_defaults  = dict(zig_params or {})   # 탐색/기본지그재그 파라미터
        self.zig_params    = dict(zig_params or {})   # 필요하면 기존 이름도 유지

        self.FIB_CFG       = dict(FIB_CFG or {})
        self.search_cfg    = dict(search_cfg or {})

        self.iters = iters; self.batch = batch
        self.T0 = T0; self.T_min = T_min

        self.retune_every = retune_every
        self.vol_window   = vol_window
        self.vol_jump     = vol_jump

        self.stoch_col    = stoch_col
        self.oversold_thr = oversold_thr
        self.auto_scale   = auto_scale

        self.best = None
        self.last_tune_pos = -1
        self.seed = seed
        self.stability = stability or {'lambda_stick': 0.5}
        
    # SelfTuner 클래스 안에 추가
    def _ensure_best_initialized(self):
        """처음 step 호출 시 self.best가 None이면 기본값 채움"""
        if getattr(self, "best", None) is None:
            zp = getattr(self, "zig_params", {}) or {}
            self.best = {
                # StochRSI D 기본
                "stochrsi_period": 14,
                "stochrsi_fastd": 3,
                # ATR 기본
                "atr_period": max(14, int(zp.get("atr_period", 21))),
                # ZigZag 기본(초기 시드)
                "zig_up_pct": float(zp.get("up_pct", 0.04)),
                "zig_down_pct": float(zp.get("down_pct", 0.04)),
                "zig_atr_mult": float(zp.get("atr_mult", 1.8)),
                "zig_min_bars": int(zp.get("min_bars", 7)),
                "zig_atr_period": int(zp.get("atr_period", 21)),
                "zig_threshold_mode": str(zp.get("threshold_mode", "max")),
            }


        
    def _need_retune(self, df):
        # 1) 초기엔 반드시 튜닝
        if self.best is None:
            return True, "first_run"
        # 2) 일정 주기마다
        if len(df) - self.last_tune_pos >= self.retune_every:
            return True, "periodic"
        # 3) 변동성 급증 감지(간단 버전)
        tail = df['close'].tail(self.vol_window).pct_change().dropna()
        if not tail.empty and tail.std() > self.vol_jump * max(1e-9, tail.rolling(10).std().mean()):
            return True, "vol_spike"
        return False, ""

    def _tune(self, df):
        windows = [
            {'sample_last': self.search_cfg.get('sample_last', 300), 'step': self.search_cfg.get('step', 10)},
            {'sample_last': max(300, self.search_cfg.get('sample_last', 300)), 'step': max(5, self.search_cfg.get('step', 10)//2)}
        ]
        best, hist_sorted = boltzmann_search_confluence(
            df, self.param_space,
            iters=self.iters, batch=self.batch, T0=self.T0, T_min=self.T_min,
            zig_params=self.zig_defaults,
            fib_ratios=self.FIB_CFG['fib_ratios'],
            fib_k=self.FIB_CFG['fib_k'], fib_mode=self.FIB_CFG['fib_mode'], fib_max_half_mult=self.FIB_CFG['fib_max_half_mult'],
            windows=windows,                 # ← 여러 윈도우
            seed=self.seed,                  # ← 고정 시드
            prior=self.best,                 # ← 이전 best (초기엔 None)
            stick_lambda=self.stability.get('lambda_stick', 0.5)
        )
        self.best = best[0]


    def _apply_best_to_df(self, df):
        bp = self.best or {}
        # RSI(고정 14)
        df['rsi'] = talib.RSI(df['close'].astype(float).values, timeperiod=14)

        # ATR: 최적값 반영
        ap = int(bp.get('atr_period', 21))
        df['atr'] = talib.ATR(
            df['high'].astype(float).values,
            df['low'].astype(float).values,
            df['close'].astype(float).values,
            timeperiod=ap
        )

        # StochRSI D: 최적값으로 강제 재계산
        sp = int(bp.get('stochrsi_period', 14))
        fd = int(bp.get('stochrsi_fastd', 3))
        ensure_stochD_inplace(
            df, col=self.stoch_col,
            rsi_period=14, stochrsi_period=sp, fastd=fd,
            force=True                           # ★ 강제
        )

        # ZigZag: 최적값 반영
        zp = dict(self.zig_defaults)
        if 'zig_up_pct'         in bp: zp['up_pct']        = float(bp['zig_up_pct'])
        if 'zig_down_pct'       in bp: zp['down_pct']      = float(bp['zig_down_pct'])
        if 'zig_atr_mult'       in bp: zp['atr_mult']      = float(bp['zig_atr_mult'])
        if 'zig_min_bars'       in bp: zp['min_bars']      = int(bp['zig_min_bars'])
        if 'zig_atr_period'     in bp: zp['atr_period']    = int(bp['zig_atr_period'])
        if 'zig_threshold_mode' in bp: zp['threshold_mode']= str(bp['zig_threshold_mode'])

        pivots, _ = zigzag_meaningful_v2(
            close=df['close'], high=df['high'], low=df['low'],
            up_pct=zp.get('up_pct',0.04), down_pct=zp.get('down_pct',0.04),
            atr_period=zp.get('atr_period',ap), atr_mult=zp.get('atr_mult',1.8),
            threshold_mode=zp.get('threshold_mode','max'),
            use_hl=True, min_bars=zp.get('min_bars',7),
            min_swing_atr=1.0, finalize_last=False
        )
        df['pivot'] = pivots
        df['pivot_price'] = np.where(df['pivot']==1, df['high'],
                              np.where(df['pivot']==-1, df['low'], np.nan))




    def step(self, df, df_, print_header=True):
        self._ensure_best_initialized()
        # SelfTuner.step(...) 내부 맨 앞, 튜닝/평가 전에
        ensure_stochD_inplace(df, col=self.stoch_col,
                              rsi_period=14,
                              stochrsi_period=int(self.best.get('stochrsi_period', 14)),
                              fastd=int(self.best.get('stochrsi_fastd', 3)),
                             force=True)

        need, reason = self._need_retune(df)
        if need:
            self._tune(df)
            if print_header:
                print(f"[AUTOTUNE] params => {self.best}  |  reason={reason}  |  idx={len(df)}")

        # 최적 파라미터를 전체 df에 반영(피벗 포함)
        self._apply_best_to_df(df)

        # (선택) 여기서 리포트까지 찍고 싶다면, 네가 만든 top_down_report를 쓰면 됨.
        try:
            res = fib_from_latest(df, pivot_col='pivot', pivot_price_col='pivot_price')
            # 일봉 리포트(원하면 활성화)
            # top_down_report(df, df_, res, d_col=self.stoch_col, oversold=20, auto_scale=self.auto_scale,
            #                 rsi_period_daily=14, rsi_period_4h=14,
            #                 use_low_4h=False,
            #                 fib_ratios=self.FIB_CFG['fib_ratios'],
            #                 fib_k=self.FIB_CFG['fib_k'],
            #                 fib_mode=self.FIB_CFG['fib_mode'],
            #                 fib_max_half_mult=self.FIB_CFG['fib_max_half_mult'])
        except Exception as e:
            print("[WARN] reporting skipped:", e)

        return dict(self.best)  # 현재 적용 파라미터 반환


## 4) TOP/DOWN report

In [8]:
def _daily_segment_window(df, seg):
    a, b = seg; ts = _ts_series(df)
    return ts.iloc[a], ts.iloc[b]

def ref_from_down_window(df_, start_ts, end_ts, *, rsi_col='rsi', use_low=False):
    ts4 = _ts_series(df_)
    sub = df_.loc[(ts4 >= start_ts) & (ts4 <= end_ts)]
    if sub.empty:
        raise ValueError("4H 구간이 비었습니다. 일봉 세그먼트 창과 4H 인덱스를 확인하세요.")
    if use_low and 'low' in sub.columns:
        idx_min = sub['low'].idxmin()
    else:
        idx_min = sub['close'].idxmin()
    return {'ref_idx': int(df_.index.get_loc(idx_min)), 'ref_ts': idx_min,
            'ref_price': float(df_.at[idx_min, 'close']), 'ref_rsi': float(df_.at[idx_min, rsi_col])}

def print_top_down_reports(df, df_, res, *, rsi_period_daily=14, rsi_period_4h=14,
                           fib_ratios=(0.786,0.618,0.5,0.382,0.236), fib_k=1.0, fib_mode='linear', fib_max_half_mult=1.2,
                           use_low_4h=False):
    bounds, atr_end = build_fib_ratio_boundaries(
        df, res, atr_col='atr', ratios=fib_ratios, k=fib_k, mode=fib_mode, max_half_mult=fib_max_half_mult
    )
    level_map = make_level_map_from_res(res, ratios=fib_ratios)

    print("\n---TOP---1d")
    seg, thr_used, reason = pick_oversold_segment_D_with_current_rule(df, d_col='close_stoch', oversold=20, auto_scale=True)
    if seg is None:
        print("[WARN] 일봉 세그먼트를 찾지 못함")
    else:
        refD = ref_from_segment_min_close(df, seg, rsi_col='rsi')
        need_reg_D  = needed_close_regular_now(df,  ref_price=refD['ref_price'], ref_rsi=refD['ref_rsi'], rsi_period=rsi_period_daily)
        hid_range_D = feasible_close_range_hidden_now(df, ref_price=refD['ref_price'], ref_rsi=refD['ref_rsi'], rsi_period=rsi_period_daily)
        print(f"[SEG] {seg}, thr_used={thr_used}, reason={reason}")
        print(f"[REF] price={refD['ref_price']:.6f}, rsi={refD['ref_rsi']:.3f}, ts={refD['ref_ts']}")
        print(f"[REG]  need_reg={need_reg_D}")
        print(f"[HID]  hid_range={hid_range_D}")
        print(f"[ATR(end)] {atr_end:.6f}")
        print_level_divergence_hits(df, bounds, need_reg=need_reg_D, hid_range=hid_range_D,
                                    rsi_period=rsi_period_daily, level_map=level_map, show_center=True)

    print("\n---DOWN---4h")
    if seg is None:
        print("[INFO] 일봉 세그먼트가 없어 DOWN 스킵") 
        return
    d_start, d_end = _daily_segment_window(df, seg)
    try:
        ref4 = ref_from_down_window(df_, d_start, d_end, rsi_col='rsi', use_low=use_low_4h)
    except Exception as e:
        print(f"[WARN] 4h 창 추출 실패: {e}"); return
    need_reg_4h  = needed_close_regular_now(df_,  ref_price=ref4['ref_price'], ref_rsi=ref4['ref_rsi'], rsi_period=rsi_period_4h)
    hid_range_4h = feasible_close_range_hidden_now(df_, ref_price=ref4['ref_price'], ref_rsi=ref4['ref_rsi'], rsi_period=rsi_period_4h)
    print(f"[SEG_D] used daily seg={seg}  window={d_start} ~ {d_end}")
    print(f"[REF_4H] price={ref4['ref_price']:.6f}, rsi={ref4['ref_rsi']:.3f}, ts={ref4['ref_ts']}")
    print(f"[REG_4H] need_reg={need_reg_4h}")
    print(f"[HID_4H] hid_range={hid_range_4h}")
    print_level_divergence_hits(df_, bounds, need_reg=need_reg_4h, hid_range=hid_range_4h,
                                rsi_period=rsi_period_4h, level_map=level_map, show_center=True)


## 5) ⚙️ Configure search / Load data

In [9]:
# df  = pd.read_csv('daily.csv', parse_dates=['datetime']).set_index('datetime')   # 예시
# df_ = pd.read_csv('4h.csv',    parse_dates=['datetime']).set_index('datetime')   # 예시

# 필수 컬럼 점검 (이미 메모리에 df/df_가 있다면 무시)
# assert {'close','high','low'}.issubset(df.columns)
# assert {'close','high','low'}.issubset(df_.columns)

SEARCH_CFG = dict(sample_last=600, step=10)

param_space = {
    # StochRSI D 튜닝(원하면 생략 가능)
    'stochrsi_period': {'type':'int','min':7,'max':34},
    'stochrsi_fastd':  {'type':'int','min':2,'max':6},

    # ATR 튜닝
    'atr_period':      {'type':'int','min':14,'max':42},

    # ZigZag 튜닝
    'zig_up_pct':        {'type':'float','min':0.03,'max':0.08},
    'zig_down_pct':      {'type':'float','min':0.03,'max':0.08},
    'zig_atr_mult':      {'type':'float','min':1.4,'max':2.8},
    'zig_min_bars':      {'type':'int','min':5,'max':12},
    'zig_atr_period':    {'type':'int','min':14,'max':42},
    'zig_threshold_mode':{'type':'choice','choices':['or','and','max']},
}


zig_params = dict(
    up_pct=0.04, down_pct=0.04,
    atr_period=21, atr_mult=1.8,
    threshold_mode='max',
    min_bars=7, min_swing_atr=1.0
)

FIB_CFG = dict(fib_ratios=(0.786,0.618,0.5,0.382,0.236),
               fib_k=1.0, fib_mode='linear', fib_max_half_mult=1.2)

print("설정 준비 완료")


설정 준비 완료


## 6) 🔍 Run Boltzmann search (find best params)

In [10]:
def _sanitize_stochd_inplace(df, col='close_stoch'):
    if col not in df.columns:
        return
    s = df[col].astype(float).copy()
    if np.isnan(s.to_numpy()).mean() > 0.2:
        s = s.interpolate(limit_direction='both')
    if np.nanmax(s.to_numpy()) <= 1.0:
        s = s * 100.0
    df[col] = s.clip(0, 100)

_sanitize_stochd_inplace(df,  'close_stoch')
_sanitize_stochd_inplace(df_, 'close_stoch')


In [13]:
# 일관성(재현성) 강화를 위한 권장 설정
# 
STABILITY = {
    'lambda_stick': 0.85,   # 이전 best에서 멀어질수록 패널티(0.4~0.8 사이 권장)
}

# 1) TOP/DOWN 확정
top_time, down_time = canonicalize_pair(top_time, down_time)


# 예: top_time / down_time은 네가 이미 정해둔 값 사용
base_space = make_param_space_for_tf(top_time)
# 3) TOP 데이터로 ATR기반 미세조정
param_space, vol_stats = atr_scaled_param_space(
    df_top.df, base_space,
    atr_period=21, lookback=400,
    updn_k_low=1.4, updn_k_high=3.2,
    mult_bias=1.0, mult_gamma=0.0,   # atr_mult는 고정에 가깝게 두고 싶으면 gamma=0
    min_bars_lo_shift=-1, min_bars_hi_shift=+2
)

tuner = SelfTuner(
    param_space=param_space,
    zig_params=zig_params,
    FIB_CFG=FIB_CFG,
    search_cfg=SEARCH_CFG,
    iters=12, batch=8, T0=0.6, T_min=0.08,
    retune_every=200, vol_window=100, vol_jump=0.55,
    stoch_col='close_stoch', oversold_thr=20, auto_scale=True,
    seed=42,                 # ← 고정 시드
    stability=STABILITY        # ← 스틱니스(안정성) 설정
)


best_now = tuner.step(df, df_)   # 자동 튜닝 + 리포트(선택)
print("현재 적용 파라미터:", best_now)


[Iter 1/12] T=0.600  best_total=-1000000000.000  batch_avg=-1000000000.000  top_prob=1.00
[Iter 2/12] T=0.510  best_total=-1000000000.000  batch_avg=-1000000000.000  top_prob=1.00
[Iter 3/12] T=0.433  best_total=-1000000000.000  batch_avg=-1000000000.000  top_prob=1.00
[Iter 4/12] T=0.368  best_total=-1000000000.000  batch_avg=-1000000000.000  top_prob=1.00
[Iter 5/12] T=0.313  best_total=-1000000000.000  batch_avg=-1000000000.000  top_prob=1.00
[Iter 6/12] T=0.266  best_total=-1000000000.000  batch_avg=-1000000000.000  top_prob=1.00
[Iter 7/12] T=0.226  best_total=-1000000000.000  batch_avg=-1000000000.000  top_prob=1.00
[Iter 8/12] T=0.192  best_total=-1000000000.000  batch_avg=-1000000000.000  top_prob=1.00
[Iter 9/12] T=0.163  best_total=-1000000000.000  batch_avg=-1000000000.000  top_prob=1.00
[Iter 10/12] T=0.139  best_total=-1000000000.000  batch_avg=-1000000000.000  top_prob=1.00
[Iter 11/12] T=0.118  best_total=-1000000000.000  batch_avg=-1000000000.000  top_prob=1.00
[Iter 12

C:\Users\hahonggu\AppData\Local\Temp\ipykernel_9236\4194910247.py:22: RuntimeWarning: All-NaN slice encountered
  if np.nanmax(d) <= 1.0:
C:\Users\hahonggu\AppData\Local\Temp\ipykernel_9236\4194910247.py:22: RuntimeWarning: All-NaN slice encountered
  if np.nanmax(d) <= 1.0:
C:\Users\hahonggu\AppData\Local\Temp\ipykernel_9236\4194910247.py:22: RuntimeWarning: All-NaN slice encountered
  if np.nanmax(d) <= 1.0:
C:\Users\hahonggu\AppData\Local\Temp\ipykernel_9236\612552976.py:7: RuntimeWarning: All-NaN slice encountered
  maxv = np.nanmax(d)
C:\Users\hahonggu\AppData\Local\Temp\ipykernel_9236\4194910247.py:22: RuntimeWarning: All-NaN slice encountered
  if np.nanmax(d) <= 1.0:
C:\Users\hahonggu\AppData\Local\Temp\ipykernel_9236\4194910247.py:22: RuntimeWarning: All-NaN slice encountered
  if np.nanmax(d) <= 1.0:
C:\Users\hahonggu\AppData\Local\Temp\ipykernel_9236\4194910247.py:22: RuntimeWarning: All-NaN slice encountered
  if np.nanmax(d) <= 1.0:
C:\Users\hahonggu\AppData\Local\Temp\ipy

## 7) ✅ Apply best params and print TOP/DOWN report

In [14]:


# --- 1) 최적 파라미터 픽업 ---
best_params = dict(best_now)  # tuner.step(...)의 반환값

ensure_stochD_inplace(df)
ensure_stochD_inplace(df_)

# --- 3) ZigZag 파라미터 튠값으로 override ---
def _tuned_zig(base, best):
    z = dict(base or {})
    if 'zig_up_pct'         in best: z['up_pct']        = float(best['zig_up_pct'])
    if 'zig_down_pct'       in best: z['down_pct']      = float(best['zig_down_pct'])
    if 'zig_atr_mult'       in best: z['atr_mult']      = float(best['zig_atr_mult'])
    if 'zig_min_bars'       in best: z['min_bars']      = int(best['zig_min_bars'])
    if 'zig_atr_period'     in best: z['atr_period']    = int(best['zig_atr_period'])
    if 'zig_threshold_mode' in best: z['threshold_mode']= str(best['zig_threshold_mode'])
    # 고정값 유지
    if 'min_swing_atr' not in z: z['min_swing_atr'] = 1.0
    return z

zp = _tuned_zig(zig_params, best_params)

# ZigZag 피벗 재계산(일봉)
pivots, _ = zigzag_meaningful_v2(
    close=df['close'], high=df['high'], low=df['low'],
    up_pct=zp.get('up_pct',0.04), down_pct=zp.get('down_pct',0.04),
    atr_period=zp.get('atr_period',21), atr_mult=zp.get('atr_mult',1.8),
    threshold_mode=zp.get('threshold_mode','max'),
    use_hl=True, min_bars=zp.get('min_bars',7), min_swing_atr=zp.get('min_swing_atr',1.0),
    finalize_last=False
)
df['pivot'] = pivots
df['pivot_price'] = np.where(df['pivot']==1, df['high'],
                      np.where(df['pivot']==-1, df['low'], np.nan))

# --- 4) 최신 스윙 피보나치 ---
res = fib_from_latest(df, pivot_col='pivot', pivot_price_col='pivot_price')

# --- 5) 최종 리포트 (TOP/DOWN 같은 형식으로 출력) ---
#  * top_down_report 를 이미 만들었을 때 그대로 사용
#  * FIB_CFG에 넣어둔 세팅을 그대로 전달

def print_params_pretty(best_params, zig_params_now, rsi_period_fixed=14):
    sp  = best_params.get('stochrsi_period', 14)
    fd  = best_params.get('stochrsi_fastd', 3)
    ap  = best_params.get('atr_period', 21)

    z   = zig_params_now
    up  = z.get('up_pct', None)
    dn  = z.get('down_pct', None)
    am  = z.get('atr_mult', None)

    def pct(x):
        return f"{x*100:.2f}%" if x is not None else "-"
    
    print(f"[ticker]              : {ticker}")
    print(f"[top_time]            : {top_time}")
    print(f"[down_time]           : {down_time}")
    
    print("\n=== Tuned Parameters ===")
    print("[Indicators]")
    print(f"  • RSI period        : {rsi_period_fixed} (fixed)")
    print(f"  • StochRSI period   : {sp}")
    print(f"  • StochRSI fastD    : {fd}")
    print(f"  • ATR period        : {ap}")

    print("\n[ZigZag]")
    print(f"  • up_pct            : {pct(up)}")
    print(f"  • down_pct          : {pct(dn)}")
    print(f"  • atr_period        : {z.get('atr_period', '-')}")
    print(f"  • atr_mult          : {am:.4f}" if am is not None else "  • atr_mult          : -")
    print(f"  • threshold_mode    : {z.get('threshold_mode','-')}")
    print(f"  • min_bars          : {z.get('min_bars','-')}")
    print(f"  • min_swing_atr     : {z.get('min_swing_atr','-')}")

# 사용 예시
print_params_pretty(best_params, zp, rsi_period_fixed=14)


# TOP/DOWN 리포트 출력
print_top_down_reports(
    df, df_, res,
    rsi_period_daily=14, rsi_period_4h=14,
    fib_ratios=(1, 0.786,0.618,0.5,0.382,0.236,0),
    fib_k=1.0, fib_mode='linear', fib_max_half_mult=1.2,
    use_low_4h=False
)


[ticker]              : ZEN/USDT
[top_time]            : 4h
[down_time]           : 1h

=== Tuned Parameters ===
[Indicators]
  • RSI period        : 14 (fixed)
  • StochRSI period   : 9
  • StochRSI fastD    : 5
  • ATR period        : 35

[ZigZag]
  • up_pct            : 5.58%
  • down_pct          : 5.09%
  • atr_period        : 32
  • atr_mult          : 1.7318
  • threshold_mode    : max
  • min_bars          : 11
  • min_swing_atr     : 1.0

---TOP---1d
[SEG] (982, 982), thr_used=20, reason=current_not_oversold_use_latest
[REF] price=9.354000, rsi=47.124, ts=982
[REG]  need_reg=None
[HID]  hid_range=(9.354009353999999, 10.289409397332152)
[ATR(end)] 0.168670

=== LEVEL × DIVERGENCE CHECK ===
R0: Price 6.21 -> [6.21 ~ 6.21] | RSI[34.61 ~ 34.61]
   · RSI→Price (boundary): 34.61→6.21 , 34.61→6.21
   - Regular : no  (need_reg=None)
   - Hidden  : no  (no-overlap)
R24: Price 6.57 -> [6.53 ~ 6.61] | RSI[35.27 ~ 35.44]
   · RSI→Price (boundary): 35.27→6.53 , 35.44→6.61
   - Regular : no